# Tutorial 1: Using ``TimeseriesExtractor``
This module is designed to perform timeseries extraction, nuisance regression, and visualization. Additionally, it
generates the necessary dictionary structure required for ``CAP``. If the BOLD images have not been preprocessed using
fMRIPrep (or a similar pipeline), the dictionary structure can be manually created.

The output in the `Extracting Timeseries` section is generated from a test run using GitHub Actions. This test uses
a truncated version of the open dataset provided by [Laumann & Poldrack](https://openfmri.org/dataset/ds000031/)
and was obtained from the OpenfMRI database, accession number ds000031.

In [ ]:
# Download packages
try:
  import neurocaps
except:
  !pip install neurocaps[windows,demo]

## Extracting Timeseries

Download test dataset used for Github Actions from Github.

In [ ]:
import os, subprocess, sys

demo_dir = "neurocaps_demo"
os.makedirs(demo_dir, exist_ok=True)

if not os.path.exists("neurocaps_demo/data/dset"):
    if sys.platform != "win32":
        cmd = """
            cd neurocaps_demo
            git clone --depth 1 --filter=blob:none --sparse https://github.com/donishadsmith/neurocaps.git
            cd neurocaps
            git sparse-checkout set tests/data/dset
            """
        os.system(cmd)
    else:
        repo_dir = os.path.join(demo_dir, "neurocaps")

        # Enable git longpath
        subprocess.run(
            ["git", "config", "--global", "core.longpaths", "true"],
            check=True,
        )

        subprocess.run(
            [
                "git",
                "clone",
                "--depth",
                "1",
                "--filter=blob:none",
                "--sparse",
                "https://github.com/donishadsmith/neurocaps.git",
            ],
            check=True,
            cwd=demo_dir,
        )

        subprocess.run(
            ["git", "sparse-checkout", "set", "tests/data/dset"],
            check=True,
            cwd=repo_dir,
        )

    # Rename folder
    os.makedirs("neurocaps_demo/data", exist_ok=True)
    os.rename("neurocaps_demo/neurocaps/tests/data/dset", "neurocaps_demo/data/dset")

Note: when an asterisk follows a name, all confounds that start with the preceding term will be automatically included.
For example, placing an asterisk after cosine (cosine*) will utilize all parameters that begin with cosine.


In [ ]:
from neurocaps.extraction import TimeseriesExtractor

confounds = ["cosine*", "a_comp_cor*", "rot*"]

parcel_approach = {"Schaefer": {"n_rois": 100, "yeo_networks": 7, "resolution_mm": 2}}

extractor = TimeseriesExtractor(
    space="MNI152NLin2009cAsym",
    parcel_approach=parcel_approach,
    standardize=True,
    use_confounds=True,
    detrend=True,
    low_pass=0.15,
    high_pass=None,
    confound_names=confounds,
    fd_threshold=0.35,
)

extractor.get_bold(
    bids_dir="neurocaps_demo/data/dset",
    session="002",
    task="rest",
    pipeline_name="fmriprep_1.0.0/fmriprep",
    tr=1.2,
    progress_bar=True,  # Parameter available in versions >= 0.21.5
)

``print`` can be used to return a string representation of the ``TimeseriesExtractor`` class.

In [ ]:
print(extractor)

The extracted timeseries is stored as a nested dictionary and can be accessed using the ``subject_timeseries``
property. The ``TimeseriesExtractor`` class has several
[properties](https://neurocaps.readthedocs.io/en/stable/generated/neurocaps.extraction.TimeseriesExtractor.html#properties)
**Some properties can also be used as setters.**

In [ ]:
print(extractor.subject_timeseries)

## Reporting Quality Control Metrics
<font size="3">Checking number of censored frames, interpolated frames, and continuous high motion stats using the `self.report_qc` method. Only censored frames with valid data on both sides are interpolated, while censored frames at the edge of the timeseries (including frames that border censored edges) are always scrubbed and counted in "Frames_Scrubbed". In the data, the last frame is the only one with an FD > 0.35. Additionally, [scipy's Cubic Spline](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CubicSpline.html) is used to only interpolate censored frames.</font>

In [ ]:
extractor.report_qc(output_dir=demo_dir, filename="qc.csv", return_df=True)

## Saving Timeseries

In [ ]:
extractor.timeseries_to_pickle(output_dir=demo_dir, filename="rest_Schaefer.pkl")

## Visualizing Timeseries

In [ ]:
# Visualizing a region
extractor.visualize_bold(subj_id="01", run="001", region="Vis")

In [ ]:
# Visualizing a several nodes
extractor.visualize_bold(subj_id="01", run="001", roi_indx=[0, 1, 2])
extractor.visualize_bold(subj_id="01", run="001", roi_indx=["LH_Vis_1", "LH_Vis_2", "LH_Vis_3"])